In [ ]:
import os
import torch
from PIL import Image

from torchvision import transforms
from networks import Generator

In [ ]:
DATA = '/home/dan/datasets/selfie2anime/testA/'
MODEL = 'models/run00_step_500000_generators.pth'
DEVICE = torch.device('cuda:0')

In [ ]:
generator = {
    'A2B': Generator(),
    'B2A': Generator()
}

generator = torch.nn.ModuleDict(generator)
generator = generator.eval().to(DEVICE)
generator.load_state_dict(torch.load(MODEL, map_location=DEVICE))

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

In [ ]:
names = [
    'female_12427.jpg', 'female_11846.jpg', 
    'female_13226.jpg', 'female_13567.jpg', 
    'female_15800.jpg', 'female_14013.jpg',
    'female_15512.jpg', 'female_4197.jpg',
    'female_12913.jpg', 'female_16169.jpg', 
    'female_14368.jpg'
]

In [ ]:
result = []

for n in names:
    
    path = os.path.join(DATA, n)
    image = Image.open(path)
    x = transform(image).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        y, _, _ = generator['A2B'](x)

    y = (255.0 * y).byte().squeeze(0).permute(1, 2, 0).cpu().numpy()
    result.append((image, Image.fromarray(y)))

In [ ]:
new = Image.new('RGB', (11 * 256, 2 * 256))

for i, (a, b) in enumerate(result):
    new.paste(a, (i * 256, 0))
    new.paste(b, (i * 256, 256))
    
new